<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex13-Binomial_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investment Analysis**, Bates, Boyer, Fletcher

# Example Chapter 13: Option Pricing with the Binomial Model

### Imports and Setup

In [1]:
import numpy as np

In the block of code below we define the function `binomial_tree`  which prices European calls and puts using the binomial tree method.  

###Function Description
`binomial_tree(type, S0, K, T, r, sigma, n)-> float`
Prices European calls and puts using the binomial tree method.   

**Inputs**

- `type (string)`: option type, must be either 'call' or 'put'.
- `S0 (float)`: the current stock price.
- `K (float)`: the strike price.
- `T (float)`: the time to maturity in years.
- `r (float)`: the annulaized risk-free rate in discrete time.
- `sigma (float)`: the anmualized volatility of the stock return.
- `n (int)`: the number of time steps in the stock tree


**Returns**  
The price of the designated option (float).


**Example Usage:**
`call_price=('call', 100, 105, 0.50, 0.01, 0.25, 3)`


In [2]:
def binomial_tree(type, S0, K, T, r, sigma, n):
    """
    Create a replicating portfolio to price a European call option using the binomial tree method.

    Parameters:
    S0 : float : Initial stock price
    K : float : Strike price
    T : float : Time to maturity (in years)
    r : float : Risk-free interest rate (annualized)
    sigma : float : Volatility of the underlying asset
    n : int : Number of time steps

    Returns:
    float : Call option price
    """

    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    discount = (1+r)**-dt

    # Initialize asset prices at maturity
    stock_prices = np.array([S0 * (u ** j) * (d ** (n - j)) for j in range(n + 1)])

    # Initialize option values at maturity
    if type=='call':
        option_values = np.maximum(stock_prices - K, 0)
    elif type=='put':
        option_values = np.maximum(K - stock_prices, 0)
    else:
        raise ValueError('Invalid option type specified. Please specify either "call" or "put".')

    # Step backwards through the tree to create the replicating portfolio
    # 1: n-1 is the starting value
    # 2: -1 is the ending value (exclusive), meaning the last value of n will be 0
    # 3: -1 is the step value, meaning we are going backwards from n-1 to 0
    for i in range(n - 1, -1, -1):
        # stock_prices[:i + 1] is the stock prices at the next time step from the lowest (index 0) to index i
        # stock_prices[:i + 1] * u is the stock prices at the current time step
        stock_prices = stock_prices[:i + 1] * u
        # option_values[1:i + 2] is the option values at the next time step from the lowest (index 1) to index i+1
        # option_values[0:i + 1] is the option values at the next time step from the lowest (index 0) to index i
        option_up = option_values[1:i + 2]
        option_down = option_values[0:i + 1]

        # Calculate the delta (number of shares)
        delta = (option_up - option_down) / (stock_prices * (u - d))

        # Calculate the bond amount (B)
        B = (option_up - delta * stock_prices * u) * discount

        # Calculate the replicating portfolio value
        option_values = delta * stock_prices + B

    return option_values[0]


In [5]:
# Example parameters
S0 = 20  # Initial stock price
K = 21  # Strike price
T = 0.25  # Time to maturity in years
r = 0.01  # Annual risk-free rate
sigma = 0.19062  # Annual volatility
n = 1  # Number of time steps

# Calculate option price using the replicating portfolio
call_price = binomial_tree('call',S0, K, T, r, sigma, n)
put_price = binomial_tree('put',S0, K, T, r, sigma, n)
print(f"Call Option Price: {call_price:.4f}")
print(f"Put Option Price: {put_price:.4f}")

Call Option Price: 0.4880
Put Option Price: 1.4358
